In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys

import matplotlib.animation as animation

from leap3d.config import DATA_DIR, DATASET_DIR, X_MIN, X_MAX, Y_MIN, Y_MAX, Z_MIN, Z_MAX, SAFETY_OFFSET, MELTING_POINT, TIMESTEP_DURATION
from leap3d.scanning import ScanParameters
from leap3d.scanning import ScanResults
from leap3d.plotting import get_frames_for_temperature_cross_section_animation, \
    get_frames_for_top_layer_temperatures, \
    animate_cross_secion_and_top_player, plot_dataset_histograms

from leap3d.laser import get_next_laser_position

In [ ]:
ANIMATION_FRAME_DURATION = 1
GIF_FPS = 60
PLOT_DIR = Path("../Plots/")

In [ ]:
case_index = 0
case_filename = DATA_DIR / "case_0000.npz"
params_filename =  DATA_DIR / "Params.npy"
rough_coord_filename =  DATA_DIR / "Rough_coord.npz"

In [ ]:
case_params = ScanParameters(params_filename, rough_coord_filename, case_index,
                             x_min=X_MIN, x_max=X_MAX,
                             y_min=Y_MIN, y_max=Y_MAX,
                             z_min=Z_MIN, z_max=Z_MAX,
                             safety_offset=SAFETY_OFFSET,
                             melting_point=MELTING_POINT,
                             timestep_duration=TIMESTEP_DURATION)
case_params

In [ ]:
case_results = ScanResults(case_filename)

In [ ]:
fig, ims = get_frames_for_top_layer_temperatures(case_results, case_params)
fig.get_axes()[0].set_title("Top layer view of temperature map over time")
ani = animation.ArtistAnimation(fig, ims, interval=ANIMATION_FRAME_DURATION, blit=True,
                                repeat_delay=1000)
ani.save(PLOT_DIR / "top_heatmap.gif", fps=GIF_FPS)

In [ ]:
fig, ims = get_frames_for_temperature_cross_section_animation(case_results, case_params)

In [ ]:
fig.get_axes()[0].set_title("Cross section of temperature map over time")
ani = animation.ArtistAnimation(fig, ims, repeat=True, interval=ANIMATION_FRAME_DURATION, blit=True, repeat_delay=1000)
ani.save(PLOT_DIR / "temperature_cross_section.gif", fps=GIF_FPS, progress_callback=print)


In [ ]:
fig, ims = animate_cross_secion_and_top_player(case_results, case_params)
ani = animation.ArtistAnimation(fig, ims, repeat=True, interval=ANIMATION_FRAME_DURATION, blit=True, repeat_delay=1000)
ani.save(PLOT_DIR / "summary.gif", fps=GIF_FPS, progress_callback=print)

In [ ]:
from torchvision.transforms import transforms
from leap3d.dataset import LEAP3DDataset

from leap3d.transforms import normalize_temperature_2d


normalize_temperature = lambda x: normalize_temperature_2d(x, melting_point=MELTING_POINT, base_temperature=0, inplace=True)

train_transforms = transforms.Compose([
        transforms.Lambda(normalize_temperature)
    ])

target_transforms = transforms.Compose([
    transforms.Lambda(normalize_temperature)
])
dataset = LEAP3DDataset(DATASET_DIR, train=True, transform=train_transforms, target_transform=target_transforms)

In [ ]:
z_scores_train, z_scores_target = dataset.get_temperature_z_scores()

In [ ]:
fig, axes = plot_dataset_histograms(dataset)
fig.set_figheight(10)
fig.set_figwidth(5)
fig.savefig(PLOT_DIR / "dataset_normalized_histogram.png")

In [ ]:
dataset = LEAP3DDataset(DATASET_DIR, train=True, transform=train_transforms, target_transform=target_transforms)

In [ ]:
fix, axes = plot_dataset_histograms(dataset)
fig.set_figheight(10)
fig.set_figwidth(5)
fig.savefig(PLOT_DIR / "dataset_histogram.png")